This is starting as a copy of the `baby-pretrain` notebook from challenge 13 but I will run it with a GPU. Look at `getting-ready.ipynb` in this folder first, then once on GPU machine run `train-tokenizer.ipynb` then run this notebook.

In [ ]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_gpt import GPTConfig, GPT
import my_nanochat.my_tokenizer
from my_nanochat.my_dataset import text_iterator
from my_nanochat.my_dataloader import tokenizing_distributed_data_loader
from my_nanochat.my_tokenizer import MyTokenizer
from my_nanochat.my_common import get_base_dir, autodetect_device_type
import torch
import math
import os
from contextlib import nullcontext

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

In [ ]:
device_type = autodetect_device_type()
device = device_type

In [ ]:
# model architecture
depth = 4
max_seq_len = 128

# training horizon
num_iterations = 1000

# optimization (not sure why this section is called that yet)
device_batch_size = 1
total_batch_size = 128 # (device_batch_size x max_seq_len)

# these next 4 are for the optimizers and we already saw them in setup_optimizers()
embedding_lr = 0.2
unembedding_lr = 0.004
weight_decay = 0.0
matrix_lr = 0.02

grad_clip = 1.0

# LR scheduler
warmup_ratio = 0.0
warmdown_ratio = 0.2
final_lr_fraction = 0.0

In [ ]:
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16) if device_type == "cuda" else nullcontext()

In [ ]:
tokenizer = my_nanochat.my_tokenizer.get_tokenizer()
vocab_size = tokenizer.get_vocab_size()
vocab_size

In [ ]:
# model kwargs are derived from desired depth of model
num_layers = depth
model_dim = depth * 64 # so for example in the default in GPTConfig it's 12 * 64 = 768)
num_heads = max(1, (model_dim + 127) // 128)
num_kv_heads = num_heads
num_layers, model_dim, num_heads, num_kv_heads

In [ ]:
# figure out the needed gradient accumulation to reach the desired total batch size
tokens_per_fwdbwd = device_batch_size * max_seq_len
grad_accum_steps = total_batch_size // tokens_per_fwdbwd
tokens_per_fwdbwd, grad_accum_steps

In [ ]:
model_config_kwargs = dict(
    sequence_len=max_seq_len,
    vocab_size=vocab_size, 
    n_layer=num_layers,
    n_head=num_heads,
    n_kv_head=num_kv_heads,
    n_embd=model_dim,
)
with torch.device("meta"):
    model_config = GPTConfig(**model_config_kwargs)
    model = GPT(model_config)
model.to_empty(device=device)

In [ ]:
model.init_weights()

In [ ]:
model.get_device()

In [ ]:
orig_model = model # original, uncompiled model -- looks like even in this minimal notebook we might use it

In [ ]:
model = torch.compile(model, dynamic=False)
model

In [ ]:
num_params = sum([param.numel() for param in model.parameters()])
num_params

In [ ]:
total_tokens = total_batch_size * num_iterations
total_tokens # total number of training tokens

In [ ]:
# initialize optimizer
optimizers = model.setup_optimizers(
    unembedding_lr=unembedding_lr,
    embedding_lr=embedding_lr,
    matrix_lr=matrix_lr,
    weight_decay=weight_decay,
)
adamw_optimizer, muon_optimizer = optimizers

In [ ]:
# initialize DataLoader
train_loader = tokenizing_distributed_data_loader(device_batch_size, max_seq_len, split="train", device=device)
x, y = next(train_loader)
x.shape, y.shape

In [ ]:
# set up hyperparameter scheulders

In [ ]:
# learning rate scheduler
def get_lr_multiplier(it):
    warmup_iters = round(warmup_ratio * num_iterations)
    warmdown_iters = round(warmdown_ratio * num_iterations)
    if it < warmup_iters:
        return (it + 1) / warmup_iters
    elif it <= num_iterations - warmdown_iters:
        return 1.0
    else:
        progress = (num_iterations - it) / warmdown_iters
        return progress * 1.0 + (1 - progress) * final_lr_fraction

def get_muon_momentum(it):
    frac = min(it / 300, 1)
    momentum = (1 - frac) * 0.85  + frac * 0.95
    return momentum

### the training loop!

In [ ]:
for step in range(num_iterations):
    for micro_step in range(grad_accum_steps):
        with autocast_ctx: # before I added this in was getting BackendCompilerFailed: backend='inductor' raised: RuntimeError: expected mat1 and mat2 to have the same dtype, but got: c10::BFloat16 != float
            loss = model(x, y)
        train_loss = loss.detach()
        loss = loss / grad_accum_steps # seems import to understand, but n/a here since grad_accum_steps is 1, see his comment
        loss.backward()
        x, y = next(train_loader)
    # gradient clipping
    if grad_clip > 0.0:
        torch.nn.utils.clip_grad_norm_(orig_model.parameters(), grad_clip) # check exactly what this does, it's not a simple cip
    # step optimizers
    lrm = get_lr_multiplier(step)
    for opt in optimizers:
        for group in opt.param_groups:
            group["lr"] = group["initial_lr"] * lrm
    muon_momentum = get_muon_momentum(step)
    for group in muon_optimizer.param_groups:
        group["momentum"] = muon_momentum
    for opt in optimizers:
        opt.step()
    model.zero_grad(set_to_none=True)

    if step % 10 == 0:
        print(f"step: {step}, loss: {train_loss}")

In [ ]:
torch.save(orig_model.state_dict(), "model.pth")

In [ ]:
!ls -lh model.pth

In [ ]:
# show top 3 next tokens for a few prompts
for prompt in ['The person', 'He went to', '1 + 2 = ', 'first of', '3 cats and 2', 'mom and', 'the red', 'She']:
    with autocast_ctx:
        logits = orig_model(torch.tensor([tokenizer.encode(prompt)], device=device)).detach()
    top_3_next_tokens = torch.topk(logits[0,-1,:], k=3).indices
    print(f"{prompt}{'|'.join([tokenizer.decode([token]) for token in top_3_next_tokens])}")